In [1]:
import nipype.pipeline.engine as pe
import nipype.interfaces.fsl as fsl
import nipype.interfaces.io as nio
import nipype.interfaces.ants as ants
import nipype.interfaces.utility as util
from nipype.interfaces.c3 import C3dAffineTool


In [2]:
import glob

In [3]:
def extract_weighted_timeseries(data_fn, mask_fn):
    import nibabel as nb
    import numpy as np
    from nipype.utils.filemanip import split_filename
    import os
    
    data_image = nb.load(data_fn)
    mask_image = nb.load(mask_fn)
    
    data = data_image.get_data()
    mask = mask_image.get_data()
 
    
    ts = (data[mask > 0] * mask[mask > 0][:, np.newaxis]  / mask.sum()).sum(0)
    
    _, fn, _ = split_filename(data_fn)
    new_fn = os.path.abspath('%s_weighted.txt' % fn)
    
    np.savetxt(new_fn, ts)
    
    return new_fn
    

In [7]:
templates = {'mean_epi':'/home/gdholla1/data/daphne/preprocessed/feat_preprocess/mean/_subject_id_{subject_id}/_fwhm_0.0/run1_dtype_mcf_mask_gms_mean.nii.gz',
              't1_weighted':'/home/gdholla1/data/daphne/clean/{subject_id}/t1.nii.gz',
             'epi2t1_weighted':'/home/gdholla1/data/daphne/registration/slab2t1_weighted/_subject_id_{subject_id}/run1_dtype_mcf_mask_gms_mean_flirt_whole_brain_epi_flirt.mat',
             'mni2t1_weighted':'/home/gdholla1/data/daphne/registration/mni2t1_weighted/_subject_id_{subject_id}/transformInverseComposite.h5',
             'filtered_epi':'/home/gdholla1/data/daphne/preprocess_phys/motion_regressors_filtered_files/_subject_id_{subject_id}/_fwhm_0.0/_addmean*/run*_maths_maths_maths_dtype_mcf_mask_gms_tempfilt_regfilt_maths.nii.gz'}

workflow = pe.Workflow(name='register2mni', base_dir='/home/gdholla1/workflow_folders/')

subject_ids = [fn.split('/')[-1] for fn in glob.glob('/home/gdholla1/data/daphne/clean/*')]

identity = pe.Node(util.IdentityInterface(fields=['subject_id']), name='identity')
identity.iterables = [('subject_id', subject_ids)]
# identity.iterables = [('subject_id', ['S00'])]
             
selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')


convert2itk = pe.Node(C3dAffineTool(),
                  name='convert2itk')
convert2itk.inputs.fsl2ras = True
convert2itk.inputs.itk_transform = True

workflow.connect(selector, 'epi2t1_weighted', convert2itk, 'transform_file')
workflow.connect(selector, 'mean_epi', convert2itk, 'source_file')
workflow.connect(selector, 't1_weighted', convert2itk, 'reference_file')

transform_merger = pe.Node(util.Merge(2), name='transform_merger')
workflow.connect(selector, 'mni2t1_weighted', transform_merger, 'in2')
workflow.connect(convert2itk, 'itk_transform', transform_merger, 'in1')

mni2mean_epi = pe.Node(ants.ApplyTransforms(), name='mni2mean_epi')
mni2mean_epi.inputs.input_image = '/usr/share/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'
mni2mean_epi.inputs.invert_transform_flags = [True, False]

workflow.connect(selector, 'mean_epi', mni2mean_epi, 'reference_image')
workflow.connect(transform_merger, 'out', mni2mean_epi, 'transforms')

mask_templates = {'mask':'/home/gdholla1/data/standard_masks/resampled_atag_masks/{mask}.nii.gz'}
mask_selector = pe.Node(nio.SelectFiles(mask_templates), name='mask_selector')

mask_selector.iterables = [('mask', ['STh_L', 'STh_R', 'SN_L', 'SN_R', 'PAG', 'RN_L', 'RN_R', 'GPi_L', 'GPi_R', 'GPe_L', 'GPe_R'][:])]

mask_transformer = pe.Node(ants.ApplyTransforms(), name='mask_transformer')
mask_transformer.inputs.invert_transform_flags = [True, False]
workflow.connect(selector, 'mean_epi', mask_transformer, 'reference_image')
workflow.connect(transform_merger, 'out', mask_transformer, 'transforms')
workflow.connect(mask_selector, 'mask', mask_transformer, 'input_image')


## EXTRACT WEIGHTED SIGNAL PHYS CONJUNCT
extract_weighted_phys = pe.MapNode(util.Function(function=extract_weighted_timeseries,
                                                 input_names=['data_fn', 'mask_fn'],
                                                 output_names=['weighted_timeseries']),
                                   iterfield=['data_fn'],
                                   name='extract_weighted_phys')

workflow.connect(selector, 'filtered_epi', extract_weighted_phys, 'data_fn')
workflow.connect(mask_transformer, 'output_image', extract_weighted_phys, 'mask_fn')

ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = '/home/gdholla1/data/daphne/mni_masks'

workflow.connect(mni2mean_epi, 'output_image', ds, 'mni2mean_epi')
workflow.connect(mask_transformer, 'output_image', ds, 'masks2mean_epi')
workflow.connect(extract_weighted_phys, 'weighted_timeseries', ds, 'extracted_signal')


workflow.run()